In [1]:
!pip3 install sentence_transformers
import numpy as np
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
# Data preparation
# Use Brown corpus
import nltk

nltk.download("brown")
from nltk.corpus import brown

sentences = [" ".join(sent) for sent in brown.sents()]

In [4]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(
    sentences=sentences, convert_to_numpy=True, show_progress_bar=True
)
query_embedding, index_embeddings = embeddings[0], embeddings[1:]

Batches:   1%|          | 16/1792 [00:14<21:00,  1.41it/s] 

In [ ]:
print(query_embedding.shape, index_embeddings.shape)

In [ ]:
# Semantic search function
def semantic_search(query_embedding, embeddings):
    similarities = cosine_similarity([query_embedding], embeddings)
    return np.argsort(-similarities[0])

In [ ]:
embedding_sizes = list(range(25, query_embedding.shape[0], 25))
times = []
n_repeat = 500

for size in embedding_sizes:
    # Generate embeddings
    query_embedding_reduced = query_embedding[:size]

    # Reduce the dimension of the query embedding to match the current size
    index_embeddings_reduced = index_embeddings[:, :size]

    # Perform semantic search and measure time
    start_time = time()
    for i in range(n_repeat):
        semantic_search(query_embedding_reduced, index_embeddings_reduced)
    end_time = time()

    time_taken = (end_time - start_time) / n_repeat
    print(f"Embedding Size: {size}, Time taken: {time_taken:.4f} seconds\n")
    times.append(time_taken)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(embedding_sizes, times, marker="o", linestyle="-", color="blue")
plt.title("Trade-off Between Embedding Size and Search Speed")
plt.xlabel("Embedding Size")
plt.ylabel("Time Taken (seconds)")
plt.grid(True)


# Display the graph
plt.show()